# Digit Recognizer - A Deep Learning Approach

## Why use deep learning?

Deep learning has substantially pushed the state-of-the-art in a wide range of tasks, including speech recognition, image recognition, and object detection. In 2012, deep neural networks (DNNs) shocked the world by halving the error rate in the ImageNet contest, and precipitated the rapid adoption of deep learning by the whole community. From then on, practitioners just don't get any result if they don't use deep learning. Therefore, let's take a look at this hottest technique.


## Why recognize digits?

Recognizing digits with MNIST dataset is the hello world example in the machine learning world. It is simple, but illustrative enough. For those who take 688, you must have got familiar with it during lab4, and we reuse some code to maintain the consistency. For those who don't, the MNIST database is a large database of 28×28 pixel handwritten digits, with 60,000 training images and 10,000 testing images.

In [ ]:
import numpy as np
import struct

# the code for loading the database into numpy array.
def parse_images(filename):
    f = open(filename,"rb");
    magic,size = struct.unpack('>ii', f.read(8))
    sx,sy = struct.unpack('>ii', f.read(8))
    X = []
    for i in range(size):
        im =  struct.unpack('B'*(sx*sy), f.read(sx*sy))
        X.append([float(x)/255.0 for x in im]);
    return np.array(X).transpose();

def parse_labels(filename):
    f = open(filename,"rb");
    magic,size = struct.unpack('>ii', f.read(8))
    return np.array(struct.unpack('B'*size, f.read(size)))

# dataset can be downloaded from http://yann.lecun.com/exdb/mnist/
train_images = parse_images("train-images-idx3-ubyte")
train_labels = parse_labels("train-labels-idx1-ubyte")
test_images = parse_images("t10k-images-idx3-ubyte")
test_labels = parse_labels("t10k-labels-idx1-ubyte")

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

# shows some samplings of the digits from the MNIST database
# such that we have a feeling of the database
M,N = 10,20
fig, ax = plt.subplots(figsize=(N,M))
digits = np.vstack([np.hstack([np.reshape(train_images[:, i*N+j],(28,28)) 
                               for j in range(N)]) for i in range(M)])
ax.imshow(255-digits, cmap=plt.get_cmap('gray'))
plt.show()

[![MNIST.png](https://s11.postimg.org/azqm5mg9f/MNIST.png)](https://postimg.org/image/8v694jemn/)

## What is deep learning?

Deep, in its appearance, means that the networks have great depth, i.e. have many layers of hidden units. 20 years ago a typical neural network (NN) had 3-5 layers of hidden units, while recently Microsoft researchers won the ImageNet contest with 152 layers NNs!$$$$However, deep learning is not just stacking layer over layer. Without modern architectures and techniques, a deeper network, with millions of parameters, may suffer from:
1. Hard to train.
2. Overfitting.

To solve these issues, scientists introduced new architectures, like convolutional networks, and new training techniques, like dropout. Therefore, DNNs in essence are NNs with modern architectures, and are trained with modern techniques. In this tutorial, we will first introduce a simple NN, and then proceed to some cool deep techniques.

## Multiclass classification and one-hot representation

Classification of the digits is a multiclass (10 classes) classification. In binary classification problem, we use 0 to represent one class, while use 1 to represent another. This can be easily extended to the multiclass scenario. Specifically, assuming we have $C$ classes, we use a binary vector of length $C$, all except one of which are zeros, hence named one-hot. Say the original label is $k$, then only the kth element of the one-hot vector is 1.

In [ ]:
# converts dense labels to one-hot labels.
def dense_to_one_hot(labels_dense, num_classes=10):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return np.transpose(labels_one_hot)

train_labels = dense_to_one_hot(train_labels)
test_labels = dense_to_one_hot(test_labels)


## Neural networks

Neural networks consist of a collection of neurons, which is a logistic unit. Mathematically, the output $y$ satisfies the following formula:$$y=\sigma(w_{1}x_{1}+w_{2}x_{2}+b)$$where $\sigma(x)$ is the sigmoid function, and is defined as $\sigma(x)=\frac{1}{1+\exp(-x)}$. The sigmoid is also called activation function, which limits the output between 0 and 1. If $y>0.5$, the neuron is activated; otherwise, it is deactivated. Below shows a single neuron.[![single_neuron.png](https://s22.postimg.org/kvfm6qhc1/single_neuron.png)](https://postimg.org/image/p4kc8wkl9/)

In [ ]:
def sigmoid(x):
    return np.exp(-np.logaddexp(0, -x)) # compute sigmoid in python

One of the most powerful ability of NNs is that they can, rather than need to be fed with manually crafted features, directly learn abstract representations from the raw input. Therefore, all we need to do is to construct a network and throw all inputs to it. In our case, we have 28×28 pixel images, and thus the size of input is 784. The output size is 10, as we have 10 classes in one-hot representation. Also, we need to have some neurons in between input and output to model the underlying mapping between pixels and classes. As a first attempt, let's use one layer of neurons and set the number of neurons as 256. (the number does not matter once it is reasonable)

For simplicity of mathematical notation, we introduce the matrix formulation of the previous equation. Now, the input $\mathbf{x}$ is a vector of 784 elements, the weight matrix $\mathbf{W_1}$ is 256×784, $\mathbf{W_2}$ is 10×256, and the output $\mathbf{y}$ is a vector of 10 elements.$$h(\mathbf{x})=\sigma(\mathbf{W_1}\mathbf{x}+\mathbf{b_1})$$$$\mathbf{y}=\sigma(\mathbf{W_2}h(\mathbf{x})+\mathbf{b_2})$$Finally, we should classify the image as the digit that is most probable. Hence, we should use maximum. However, we instead use soft maximum function here, as it is easier to compute its gradient, which is defined as:$$p(y=i)=\exp(y_i)/\sum_{j=0}^{C}{\exp(y_j)}$$Intuitively, the softmax function outputs a number between 0 and 1, which can be viewed as the probability of the image belonging to a certain class. For better understanding, the architecture of the network is illustrated below (bias is omitted).[![one_layer_network.png](https://s22.postimg.org/4yt40k4ap/one_layer_network.png)](https://postimg.org/image/uucujr64d/)

In [ ]:
# softmax layer
def softmax(x):
    exp_x = np.exp(x)
    sum_exp_x = np.sum(exp_x, 0)
    return np.divide(exp_x, sum_exp_x)

# feed-forward phase of neural networks
def feed_forward(x, W1, b1, W2, b2):
    h1 = sigmoid(np.add(np.matmul(W1, x), b1))
    y = sigmoid(np.add(np.matmul(W2, h1), b2))
    p = softmax(y)
    return h1, y, p

Great! We have officially finished the so-called feed-forward process of NNs. Now, we have many adjustable parameters, $\mathbf{W_1}$, $\mathbf{W_2}$, $\mathbf{b_1}$, and $\mathbf{b_2}$, it is time to learn these parameters. 

Imagine that the network outputs a vector (0.1, 0.1, ..., 0.1) for a certain image, while the label for this image is 0. Thus, the one-hot representation is (1, 0, 0, ..., 0). Certainly, we want the network's output to match the one-hot representation, i.e. increase the probability of 0, and make other digits less likely. Thus, we hope to change the output $\mathbf{y}$ by $(\Delta{y},-\Delta{y},-\Delta{y},...,-\Delta{y})$, where $\Delta{y}$ is a small positive number, by changing individual parameters, say $b_1$. Apparently, this is where gradient should come into the scene. Specifically, the new $b_1$ should be set as something like:$$b_1=b_1+\alpha\frac{\partial{\Delta\mathbf{y}}}{\partial{b_1}}$$where $\alpha$ is a hyper-parameter we should decide later. Also, $\alpha$ controls how far each step is, which is known as the learning rate. 

Until now the $\Delta{y}$ part is just intuition, we want to formalize it mathematically. As we mentioned before, the output of softmax function is the probability that one image belongs to a certain class. Therefore, we can construct a MLE estimator, and we hope to maximize the likelihood. Given a certain image, we obtain:$$p_{MLE}=\prod_{i=0}^{C}{p_i}^{t_i}$$where $t_{k}=1$ if $k$ is the true label. Note that when $t_i=0$, ${p_i}^{t_i}$ gives a one, and when $t_{i}=1$, it gives $p_i$. Thus, this is equivalent to maximize $p_k$, where $k$ is the correct label. However, this mathematical notation makes the manipulation simpler. Moreover, equivalently, it is usually more convenient to minimize the negative log-likelihood, and we call it loss function. Also, this famous loss function is named as cross-entropy loss.$$L=-\sum_{i=0}^{C}{t_i}\log{p_i}$$Therefore, instead of using $\Delta\mathbf{y}$, formally, we have the update rule as:$$\mathbf{W}=\mathbf{W}-\alpha\frac{\partial{L}}{\partial{\mathbf{W}}}$$$$\mathbf{b}=\mathbf{b}-\alpha\frac{\partial{L}}{\partial{\mathbf{b}}}$$Since $L$ is a function of $\mathbf{p}$ and $\mathbf{p}$ is function of $\mathbf{y}$, we denote as $L=o(\mathbf{y})$. According to the chain rule in calculus, it is easy to get$$\frac{\partial{L}}{\partial{\mathbf{W_2}}}=\frac{\partial{L}}{\partial{\mathbf{y}}}\frac{\partial{\mathbf{y}}}{\partial{\mathbf{W_2}}}$$$$\frac{\partial{L}}{\partial{\mathbf{b_2}}}=\frac{\partial{L}}{\partial{\mathbf{y}}}\frac{\partial{\mathbf{y}}}{\partial{\mathbf{b_2}}}$$$$\frac{\partial{L}}{\partial{\mathbf{W_1}}}=\frac{\partial{L}}{\partial{\mathbf{y}}}\frac{\partial{\mathbf{y}}}{\partial{h(\mathbf{x})}}\frac{\partial{h(\mathbf{x})}}{\partial{\mathbf{W_1}}}$$$$\frac{\partial{L}}{\partial{\mathbf{b_1}}}=\frac{\partial{L}}{\partial{\mathbf{y}}}\frac{\partial{\mathbf{y}}}{\partial{h(\mathbf{x})}}\frac{\partial{h(\mathbf{x})}}{\partial{\mathbf{b_1}}}$$Think of this procedure: we first have the loss w.r.t $\mathbf{y}$, which is $\frac{\partial{L}}{\partial{\mathbf{y}}}$. Then, we can calculate the loss w.r.t. $h(\mathbf{x})$, which is $\frac{\partial{L}}{\partial{\mathbf{y}}}\frac{\partial{\mathbf{y}}}{\partial{h(\mathbf{x})}}$. Finally, based on the loss w.r.t. $h(\mathbf{x})$, we can compute the loss w.r.t. $\mathbf{W_1}$ and $\mathbf{b_1}$. The process looks like we are propagating the loss from $L$ to $\mathbf{y}$ to $h(\mathbf{x})$ and finally to every parameter, and this is exactly why this learning process is named back-propagation.

With simple calculus and some mathematical manipulation, we can get the value of each component for the above rules. To save some space, here list some important results as an example:$$\frac{\partial{L}}{\partial{\mathbf{y}}}=\mathbf{p}-\mathbf{t}$$
$$\frac{\partial{\mathbf{y}}}{\partial{\mathbf{b_1}}}=\mathbf{y}(1-\mathbf{y})$$
$$\frac{\partial{\mathbf{y}}}{\partial{h(\mathbf{x})}}=\mathbf{W_2}^T\mathbf{y}(1-\mathbf{y})$$Interestingly, the loss w.r.t. $\mathbf{y}$ is $\mathbf{p}-\mathbf{t}$, which matches our intuition. Thus, we can view loss functions as a mathematical formulation of our intuition.

The above is the famous optimization technique gradient descent (GD). One thing left is that we need to repeatedly apply these update rules until the loss function reaches a local minimum (Optimization of NNs is non-convex, and this is partly why we should use GD). Basically, the loss on the training set will keep going down, but we actually desire to have a higher accuracy on the test set. They are not equivalent. One thing is overfitting. Therefore, we hope to stop before the error rate on the test set go up. A common approach is to specify a maximum amount of steps for taking gradient, which can be selected based on cross-validation. 

Before we proceed to training, we first need to figure out some ways to evaluate our approach: usually we use the loss and error rate on both the training set and the test set.

In [ ]:
# the evaluation code
def cross_entropy(y, truth):
    entropy = -np.add(np.multiply(truth, np.log(y)), \
                  np.multiply(-np.subtract(truth, 1), np.log(-np.subtract(y, 1))))
    mean_entropy = np.sum(entropy) / entropy.shape[1]
    return mean_entropy

def error_rate(y, truth):
    return np.mean(np.argmax(truth, 0) != np.argmax(y, 0))

def evaluate(y, truth):
    loss = cross_entropy(p, truth)
    error = error_rate(p, truth)
    return loss, error

Finally, we can start our training code. Note that, instead of computing gradient for each example and summing them, we concatenate the examples together, such that $\mathbf{x}$ becomes 784×60,000, and we can use matrix multiplication, which is much faster. For now, we have two hyper-parameters, learning rate and epochs, and we, somewhat randomly, set them as 1e-4 and 100 in the following code.

In [ ]:
def weight_variables(dim1, dim2, variance):
    matrix = np.multiply(2, np.random.rand(dim1, dim2))
    matrix = np.multiply(np.subtract(matrix, 1), variance)
    return matrix

def bias_variables(dim1, dim2, variance):
    matrix = np.ones((dim1, dim2))
    matrix = np.multiply(matrix, variance)
    return matrix

import math

num_hidden_units = 256
learn_rate = 1e-4
epochs = 200

images = train_images
labels = train_labels

variance1 = math.sqrt(6.0) / (784.0 + num_hidden_units)
W1 = weight_variables(num_hidden_units, 784, variance1)
b1 = bias_variables(num_hidden_units, 1, 0)

variance2 = math.sqrt(6.0) / (10.0 + num_hidden_units)
W2 = weight_variables(10, num_hidden_units, variance2)
b2 = bias_variables(10, 1, 0)

images_transposed = np.transpose(images)
images_ones = np.ones((images_transposed.shape[0], 1))

for epoch in range(0, epochs):
    h1, y, p = feed_forward(images, W1, b1, W2, b2)

    # every 20 steps print train loss and error rate
    if epoch % 20 == 0:
        train_loss, train_error = evaluate(p, train_labels)
        print "training loss", train_loss, "training error:", train_error
    
    h1_tranposed = np.transpose(h1)
    
    loss_derivative = np.subtract(p, labels)
    output = np.multiply(loss_derivative, np.multiply(y, (1-y)))
    hidden = np.multiply(np.matmul(np.transpose(W2), output), np.multiply(h1, (1-h1)))

    W2_gradient = np.matmul(output, h1_tranposed)
    b2_gradient = np.matmul(output, np.ones((h1_tranposed.shape[0], 1)))
    W2 = np.subtract(W2, np.multiply(learn_rate, W2_gradient))
    b2 = np.subtract(b2, np.multiply(learn_rate, b2_gradient))

    W1_gradient = np.matmul(hidden, images_transposed)
    b1_gradient = np.matmul(hidden, images_ones)
    W1 = np.subtract(W1, np.multiply(learn_rate, W1_gradient))
    b1 = np.subtract(b1, np.multiply(learn_rate, b1_gradient))

_, _, p = feed_forward(train_images, W1, b1, W2, b2)
train_loss, train_error = evaluate(p, train_labels)
print "training loss", train_loss, "training error:", train_error
_, _, p = feed_forward(test_images, W1, b1, W2, b2)
test_loss, test_error = evaluate(p, test_labels)
print "test loss", test_loss, "test error:", test_error

## Going deeper

Frankly, the above result is pretty crappy in NNs' world. To make it better, we will introduce several more advanced techniques in this section.

### Stochastic gradient descent

Gradient descent calculates perfect gradient of the training set, but the gradient may not model the test set well. Stochastic gradient descent (SGD), as its name suggests, introduces some stochasticity to the model, which potentially lessens the overfitting. Here, we will use a mini-batch SGD, i.e. every time we use a mini-batch of size $N$ to calculate the gradient. This can utilize the matrix multiplication, which is one of the most optimized code in the world, and thus the computation is much more efficient. Also, mini-batch SGD gives less stochasticity than SGD, but more than GD. This strikes a balance between the two methods, which is proven to be very effective in practice.

In [ ]:
# get a random batch from training set of batch size
def get_batch(batch_size=50):
    indexes = np.random.randint(0, 60000, batch_size)
    return train_images[:, indexes], train_labels[:, indexes]

### Initialization

Sigmoid function is a S-shape function, and has a dynamic range. Typicall, the dynamic range is set as $[-\sqrt{3},\sqrt{3}]$. Outside the range, the neuron may always output 1 or 0, which causes the loss of capacity. Thus, we should initialize the weights in this range.

In [ ]:
# Illustration of the dynamic range of sigmoid function
%matplotlib inline
plt.figure(figsize=(8,1.5))
x = np.arange(-4,4,0.05)
y = sigmoid(x)
plt.plot(x, y)
ones = np.ones((11, 1))
straight = np.arange(0,1.1,0.1)
plt.plot(ones * np.sqrt(3), straight, 'r--')
plt.plot(ones * -np.sqrt(3), straight, 'r--')
plt.show()

[![sigmoid_range.png](https://s22.postimg.org/3xk53bx75/sigmoid_range.png)](https://postimg.org/image/5clps1ya5/)

### Weight decay and maximum norm

Similar reasons as above, we don't want the weight to become too large. The first way is to add a weight decay term to the loss function, i.e.$$L=-\sum_{i=0}^{C}{t_i}\log{p_i}+\lambda\sum{\mathbf{W}^2}$$where $\lambda$ controls the extent of weight decay. From this loss function, we know that we want to push the weights towards zero. Note that, formally, weight decay can be modelled as a MAP estimator, where the weight decay is a Gaussian prior. Specifically,$$p_{MAP}=p(\mathbf{W})\prod_{i=0}^{C}{p_i}^{t_i}$$where $p(\mathbf{W})$ is a zero mean Gaussian distribution. The second way is to constraint the maximum value directly, i.e. $\Vert\mathbf{W}\Vert\leq2$. A justification is that constraining weight vectors to lie inside a ball of fixed radius makes it possible to use a huge learning rate without the possibility of weights blowing up.

In [ ]:
# code to limit maximum norm of weights as C
def max_norm(W, C):
    W[W > C] = C
    W[W < -C] = -C
    return W

### Momentum

Optimization is prone to get stuck in local optima. The momentum solve this problem by the idea that we can escape the local optimum by a push if it is a shallow optimum, and momentum is just that push. Instead of using only current gradient, we also take into account the gradient in the previous steps. Formally, the gradient considering momentum $\hat{\Delta}^{(t)}$ is$$\hat{\Delta}^{(t)}=\Delta^{(t)}+\gamma\hat{\Delta}^{(t-1)}$$where $\gamma$ is the strength of momentum and $\Delta^{(t)}$ is the current gradient of loss function $L$.

### Dropout

Another shortcoming of current back-propagation algorithm is that all neurons are updated together. One way to break the symmetry is that we use random initialization. Another is the dropout. The idea is that, during training, we randomly disable some neurons. Specifically, we treat those disabled neurons as non-existent, and some steps remain exact the same. Therefore, instead of training one NN, we are actually training a bunch of NNs, and a subset of neurons should be able to independently predict the result. Finally, in testing, we multiply the output of all neurons with a constant 1 - dropout to offset the dropout in training.

In [ ]:
def gen_mask(dim, dropout, train=True):
    # the mask for dropout during train and test, respectively
    if train:
        mask = np.random.rand(dim, 1)
        mask[mask < dropout] = 0
        mask[mask >= dropout] = 1
    else:
        mask = (1 - dropout) * np.ones((dim, 1)) 
    return mask

def feed_forward_with_dropout(x, W1, b1, W2, b2, W3, b3, dropout, dim, train=True):
    mask = gen_mask(dim, dropout, train)
    h1 = np.multiply(mask, sigmoid(np.add(np.matmul(W1, x), b1)))
    h2 = np.multiply(mask, sigmoid(np.add(np.matmul(W2, h1), b2)))
    y = sigmoid(np.add(np.matmul(W3, h2), b3))
    p = softmax(y)
    return h1, h2, y, p

With these advanced methods in hand, we can run a much larger network. Now, let's increase hidden layers to 2 layers with 256 neurons per layer. The hyper-parameters are chosen based on our experience.

In [ ]:
np.random.seed(15388) # fix the seed

num_hidden_units1 = 256
num_hidden_units2 = 256

variance1 = math.sqrt(6.0) / (784.0 + num_hidden_units1)
W1 = weight_variables(num_hidden_units1, 784, variance1)
b1 = bias_variables(num_hidden_units1, 1, variance1)

variance2 = math.sqrt(6.0) / (num_hidden_units1 + num_hidden_units2)
W2 = weight_variables(num_hidden_units2, num_hidden_units1, variance2)
b2 = bias_variables(num_hidden_units2, 1, variance2)

variance3 = math.sqrt(6.0) / (10.0 + num_hidden_units2)
W3 = weight_variables(10, num_hidden_units2, variance3)
b3 = bias_variables(10, 1, variance3)

learn_rate = 0.01
steps = 60000 # train on one mini-batch is one step
momentum = 0.95
dropout = 0.5
weight_decay = 0.0
C = 2

last_W3 = np.zeros(W3.shape)
last_b3 = np.zeros(b3.shape)
last_W2 = np.zeros(W2.shape)
last_b2 = np.zeros(b2.shape)
last_W1 = np.zeros(W1.shape)
last_b1 = np.zeros(b1.shape)

for step in range(0, steps):
    images, labels = get_batch()
    h1, h2, y, p = feed_forward_with_dropout(
        images, W1, b1, W2, b2, W3, b3, dropout, num_hidden_units1)

    images_transposed = np.transpose(images)
    images_ones = np.ones((images_transposed.shape[0], 1))
    
    # every 5000 steps print train loss and error rate
    if step % 20000 == 0:
        train_loss, train_error = evaluate(p, labels)
        print "step", step, "training loss", train_loss, "training error:", train_error
    
    h1_tranposed = np.transpose(h1)
    h2_tranposed = np.transpose(h2)
    
    loss_derivative = np.subtract(p, labels)
    output = np.multiply(loss_derivative, np.multiply(y, (1-y)))
    hidden2 = np.multiply(np.matmul(np.transpose(W3), output), np.multiply(h2, (1-h2)))
    hidden1 = np.multiply(np.matmul(np.transpose(W2), hidden2), np.multiply(h1, (1-h1)))

    W3_gradient = np.matmul(output, h2_tranposed) \
        + (momentum * last_W3) + weight_decay * W3
    b3_gradient = np.matmul(output, np.ones((h2_tranposed.shape[0], 1))) \
        + (momentum * last_b3)
    W3 = np.subtract(W3, np.multiply(learn_rate, W3_gradient))
    b3 = np.subtract(b3, np.multiply(learn_rate, b3_gradient))

    W2_gradient = np.matmul(hidden2, h1_tranposed) \
        + (momentum * last_W2) + weight_decay * W2
    b2_gradient = np.matmul(hidden2, np.ones((h1_tranposed.shape[0], 1))) \
        + (momentum * last_b2)
    W2 = np.subtract(W2, np.multiply(learn_rate, W2_gradient))
    b2 = np.subtract(b2, np.multiply(learn_rate, b2_gradient))

    W1_gradient = np.matmul(hidden1, images_transposed) \
        + (momentum * last_W1) + weight_decay * W1
    b1_gradient = np.matmul(hidden1, images_ones) \
        + (momentum * last_b1)
    W1 = np.subtract(W1, np.multiply(learn_rate, W1_gradient))
    b1 = np.subtract(b1, np.multiply(learn_rate, b1_gradient))
    
    W3 = max_norm(W3, C)
    W2 = max_norm(W2, C)
    W1 = max_norm(W1, C)
    
    last_W3 = W3_gradient
    last_b3 = b3_gradient
    last_W2 = W2_gradient
    last_b2 = b2_gradient
    last_W1 = W1_gradient
    last_b1 = b1_gradient
    
_, _, _, p = feed_forward_with_dropout(
    train_images, W1, b1, W2, b2, W3, b3, dropout, num_hidden_units1, False)
train_loss, train_error = evaluate(p, train_labels)
print "training loss", train_loss, "training error:", train_error
_, _, _, p = feed_forward_with_dropout(
    test_images, W1, b1, W2, b2, W3, b3, dropout, num_hidden_units1, False)
test_loss, test_error = evaluate(p, test_labels)
print "test loss", test_loss, "test error:", test_error

We have a significant increase of accuracy from 84% to 98%, and the code runs faster. However, the result is far from the state-of-the-art (Nowadays, 1% error rate is very common), as this is only a tutorial and running a too large network consumes too much time. If we increase the network size and keep training, we probably will get much better result. Besides, there are many more interesting techniques in deep learning, but they are out of this simple tutorial's scope.

Finally, Hugo Larochelle hosts a great introductory course about feed-forward NNs and training NN on the website http://info.usherbrooke.ca/hlarochelle/neural_networks/content.html. You can watch some videos if you want to learn more. Hope you have a sense of the deep learning now, and thank you all for reading.